### Part 2: Modern Portfolio Theory *[15 points]*

In Lecture 7, we learn how to conduct backtesting and solve for optimal portfolios numerically. Now, you have to compare the **out-of-sample performance** of **analytical optimal portfolios** and **numerical constrained portfolios**. Your code should do the following:

1. Using **Fama-French 17 Industries Portfolios from 1980 until 2021**, conduct back-testing on the following strategies *[12.5 points]*:
    * **MSRP (analytical)** *[2.5 points]*;
    * **GMVP (analytical)** *[2.5 points]*; 
    * **MSRP no short-selling (numerical)** *[2.5 points]*; 
    * **GMVP no short-selling (numerical)** *[2.5 points]*; 
    * **Equal-weighted (EW) portfolio** *[2.5 points]*.
2. Plot the **cumulative returns** of **all five strategies** *[2.5 points]*.

#### Bonus *[15 points]*:
1. **Apply shrinkage method** to estimate **expected returns** and **variance-covariance matrices** *[5 points]*.
2. **Reduce computation time** through **parallel computing** *[10 points]*.

In [1]:
import numpy as np
import pandas as pd
import pandas_datareader as pdr
from numpy.linalg import inv
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import seaborn as sns
cmap = sns.color_palette() # plaette: 調色板

In [2]:
factor = pdr.get_data_famafrench('F-F_Research_Data_Factors', start='1-1-1926')
asset = pdr.get_data_famafrench('17_Industry_Portfolios', start='1-1-1926')
start_year = '1980'
end_year = '2021'

In [4]:
asset[0].shape # 17 industries

(1144, 17)

In [5]:
# 取得月度 factor 資料 與 月度報酬資料
df_FF = factor[0].loc[start_year:end_year]
df_R = asset[0].loc[start_year:end_year] 

In [6]:
# ER = Excessive Return = Return - Risk free return
df_ER = df_R.subtract(df_FF.RF,axis=0).shift(1) # lag one month

In [7]:
#  analytical closed-form
def gmvp(S):
    ONE = np.ones(len(S))
    return (inv(S) @ ONE) / (ONE.T @ inv(S) @ ONE)

def msrp(ER, S):
    ONE = np.ones(len(S))
    return (inv(S) @ ER) / (ONE.T @ inv(S) @ ER)
        
def pret(w, R):
    return (w @ R)

### MSRP、GMVP、EW portfolios:

In [8]:
WSize = 60 # 以 60 個月 rolling
AvgER_rolling = df_ER.rolling(WSize).mean()
CovER_rolling = df_ER.rolling(WSize).cov()

Backtest_R = pd.DataFrame(index=df_R.index,columns=['GMVP','MSRP','EW'])

for d in df_ER.index:
    ER = AvgER_rolling.loc[d]
    S = CovER_rolling.loc[d]
    R = df_R.loc[d]
    Backtest_R.loc[d,'GMVP'] = pret(gmvp(S), R)
    Backtest_R.loc[d,'MSRP'] = pret(msrp(ER, S), R)

Backtest_R.loc[:,'EW'] = df_R.mean(axis=1) 
# 增加 equally weighted 的績效，recall df_R column 存放各產業的報酬

### MSRP no short-selling、GMVP no short-selling: